In [18]:
import pandas as pd
import time
import json
import time

In [2]:
%% time
data=[]
with open('a.txt',"r") as f:
     for i,line in enumerate(f.readlines()):
            res=json.loads(line.strip())
            if len(res)==13:
                data.append(res)

# col=["remote_addr", "time_iso8601", "status", "file", "body_bytes_sent", "request_time", "uuid", "http_referer", "http_user_agent", "http_x_forwarded_for", "pno", "ld", "auth_ret"]             
# df=pd.DataFrame(data,columns=col)
# del data

In [3]:
# data=[]
with open('a-test.txt',"r") as f:
     for i,line in enumerate(f.readlines()):
            res=json.loads(line.strip())
            if len(res)==12:
                res.append("-1")
                data.append(res)
col=["remote_addr", "time_iso8601", "status", "file", "body_bytes_sent", "request_time", "uuid", "http_referer", "http_user_agent", "http_x_forwarded_for", "pno", "ld", "auth_ret"]                     


In [4]:
dd=pd.DataFrame(data,columns=col)
del data

In [5]:
df=dd[:2000]

In [21]:
def get_file_day(x):
    tem=x.split("/")
    for i,j in enumerate(tem):
        if j[:2]=="20":
            try:
                date=tem[i]+"-"+tem[i+1]+"-"+tem[i+2].split("_")[0]
                time.strptime(date, "%Y-%m-%d")
                return date
            except:
                return 2
    return 
        
        
dd['file_day1']=dd['file'].apply(get_file_day)

In [24]:
def get_file_day2(x):
    tem=x.split("/")
    for i,j in enumerate(tem):
        if len(j)>15:
            try:
                cc=j.split("_")[1]
                date=cc[:4]+"-"+cc[4:6]+"-"+cc[6:]
                time.strptime(date, "%Y-%m-%d")
                return date
            except:
                return 
    return 
        
        
dd['file_day2']=dd['file'].apply(get_file_day2)

In [26]:
def get_file_type(x):
    tem=x.split("/")
    return tem[-2].split("_")[-1]
    
    

dd['file_type']=dd['file'].apply(get_file_type)

In [27]:
dd['file_type']=dd['file_type'].apply(lambda x:  "N" if len(x)>10 else x)

In [28]:
col=dd.columns.tolist()
col.remove('file')
col.remove("uuid")

In [29]:
dd=dd[col]

In [30]:
dd["day"]=dd["time_iso8601"].apply(lambda x:x[:10])

In [31]:
dd["H"]=dd["time_iso8601"].apply(lambda x:x[11:13])

In [32]:
dd["time"]=dd["time_iso8601"].apply(lambda x:x[11:19])

In [33]:
dd["minute"]=dd["time_iso8601"].apply(lambda x:x[11:16])

In [34]:
dd['http_x_forwarded_for']=dd['http_x_forwarded_for'].apply(lambda x:0 if x=="-" else 1)

In [25]:
dd['http_x_forwarded_for'].nunique()

2570597

In [35]:
dd=dd.drop(["time_iso8601"],axis=1)

In [36]:
##count特征
for i in ['H',"minute","day","file_day1","file_day2",'http_user_agent',"http_referer","remote_addr","time"]:
    col=i+"count"
    dd[col]=dd[i].map(dd.groupby([i])['pno'].count())

In [37]:
dd=dd.drop(["http_user_agent","remote_addr"],axis=1)

In [ ]:
dd["day=day1"]=(dd['day']==dd['file_day1']).astype(int)
dd["day=day2"]=(dd['day']==dd['file_day2']).astype(int)
dd["day1=day2"]=(dd['file_day2']==dd['file_day1']).astype(int)



In [39]:
dd['auth_ret']=dd['auth_ret'].astype(int)
dd['request_time']=dd['request_time'].astype(float)

In [41]:
for i in dd.columns:
    if dd[i].dtype=="object":
        dd[i]=dd[i].astype('category')

In [1]:

fea=[col for col in dd.columns if col not in ['auth_ret']]

In [2]:
for i in dd.columns:
    if dd[i].dtype=="object":
        dd[i]=dd[i].astype('category')

In [3]:
import time
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
import argparse
import tqdm
from sklearn.metrics import roc_auc_score

In [4]:
parameters = {
    'learning_rate': 0.03,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 32,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'seed': 2020,
    'bagging_seed': 1,
    'feature_fraction_seed': 7,
    'min_data_in_leaf': 20,
    'n_jobs': -1, 
    'verbose': -1,
}

In [5]:
skf = StratifiedKFold(n_splits=5, random_state=1996, shuffle=True)
X_train=dd[dd['auth_ret']!=-1].reset_index(drop=True)
X_test=dd[dd['auth_ret']==-1].reset_index(drop=True)
del dd

y=X_train.auth_ret
X_train=X_train[fea]
X_test=X_test[fea]

oof = np.zeros(X_train.shape[0])
prediction = np.zeros(X_test.shape[0])

# del X_train
# del X_test




for index, (train_index, test_index) in enumerate(skf.split(X_train, y)):
    train_x, test_x, train_y, test_y = X_train.iloc[train_index], X_train.iloc[
            test_index], y.iloc[train_index], y.iloc[test_index]
    dtrain = lgb.Dataset(train_x, label=train_y)
    dval = lgb.Dataset(test_x, label=test_y)
    lgb_model = lgb.train(
            parameters,
            dtrain,
            num_boost_round=10000,
            valid_sets=[dval],
            early_stopping_rounds=50,
            verbose_eval=50,
            #feval=myeval,
        )
    x2 = lgb_model.predict(test_x, num_iteration=lgb_model.best_iteration)
    y2 = lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)
    oof[test_index] += x2
    prediction+= y2 / 5
roc_auc_score(y,oof)

d:\python\lib\site-packages\lightgbm\basic.py:681: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 50 rounds.
[50]	valid_0's auc: 0.722204
[100]	valid_0's auc: 0.731251
[150]	valid_0's auc: 0.737585
[200]	valid_0's auc: 0.741632
[250]	valid_0's auc: 0.744458
[300]	valid_0's auc: 0.746126
[350]	valid_0's auc: 0.747191
[400]	valid_0's auc: 0.747946
[450]	valid_0's auc: 0.748561
[500]	valid_0's auc: 0.749098
[550]	valid_0's auc: 0.749528
[600]	valid_0's auc: 0.74988
[650]	valid_0's auc: 0.750202
[700]	valid_0's auc: 0.75055
[750]	valid_0's auc: 0.750811
[800]	valid_0's auc: 0.751123
[850]	valid_0's auc: 0.751348
[900]	valid_0's auc: 0.751484
[950]	valid_0's auc: 0.751755
[1000]	valid_0's auc: 0.752012
[1050]	valid_0's auc: 0.752098
[1100]	valid_0's auc: 0.75225
[1150]	valid_0's auc: 0.752458
[1200]	valid_0's auc: 0.752538
[1250]	valid_0's auc: 0.752745
[1300]	valid_0's auc: 0.752805
[1350]	valid_0's auc: 0.752849
[1400]	valid_0's auc: 0.752957
[1450]	valid_0's auc: 0.753115
[1500]	valid_0's auc: 0.75314
[1550]	valid_0's auc: 0.753195
[

[2500]	valid_0's auc: 0.755296
[2550]	valid_0's auc: 0.755396
[2600]	valid_0's auc: 0.755392
[2650]	valid_0's auc: 0.755419
[2700]	valid_0's auc: 0.755423
Early stopping, best iteration is:
[2673]	valid_0's auc: 0.755433


0.7548144126643744